In [23]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm 
from pymongo import MongoClient

In [24]:
# Connect to MongoDB
client =  MongoClient("mongodb://localhost:27017")
db = client['B-Masseys']

In [25]:
rank_collection = db['rank']
rank_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'B-Masseys'), 'rank')

In [26]:
df = pd.read_csv('../data/processed/driver_scores.csv')

In [27]:
df

,Unnamed: 0,raceId,driverId,Grid,Position,Points,laps,Milliseconds,Fastest Lap,Rank,Fastest Lap Time,Fastest Lap Speed,statusId,Win or Loss,code,D.O.B,Nationality,Full Name
0,0,18,1,1,1,10.0,58,5690.616,39,2,87.452,218.300,1,win,HAM,1985-01-07,British,Lewis Hamilton
1,1,18,2,5,2,8.0,58,5696.094,41,3,87.739,217.586,1,lost,HEI,1977-05-10,German,Nick Heidfeld
2,2,18,3,7,3,6.0,58,5698.779,41,5,88.090,216.719,1,lost,ROS,1985-06-27,German,Nico Rosberg
3,3,18,4,11,4,5.0,58,5707.797,58,7,88.603,215.464,1,lost,ALO,1981-07-29,Spanish,Fernando Alonso
4,4,18,5,3,5,4.0,58,5708.630,43,1,87.418,218.385,1,lost,KOV,1981-10-19,Finnish,Heikki Kovalainen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24855,24855,1042,825,19,16,0.0,65,0.000,63,16,81.460,205.632,11,lost,MAG,1992-10-05,Danish,Kevin Magnussen
24856,24856,1042,154,18,17,0.0,65,0.000,59,17,81.664,205.118,11,lost,GRO,1986-04-17,French,Romain Grosjean
24857,24857,1042,849,20,18,0.0,64,0.000,53,19,81.859,204.629,12,lost,LAT,1995-06-29,Canadian,Nicholas Latifi
24858,24858,1042,826,13,19,0.0,64,0.000,59,8,80.449,208.216,12,lost,KVY,1994-04-26,Russian,Daniil Kvyat


In [28]:
df.drop(columns=['Grid', 'laps', 'Position', 'Points', 'Fastest Lap', 'D.O.B', 'Nationality', 'Unnamed: 0', 'Win or Loss', 'Milliseconds', 'code', 'Fastest Lap Speed', 'statusId'], inplace = True) 

In [29]:
race_df = pd.read_csv('../data/raw/races.csv')
race_df

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
...,...,...,...,...,...,...,...,...
1030,1043,2020,13,21,Emilia Romagna Grand Prix,2020-11-01,12:10:00,https://en.wikipedia.org/wiki/2020_Emilia_Roma...
1031,1044,2020,14,5,Turkish Grand Prix,2020-11-15,10:10:00,https://en.wikipedia.org/wiki/2020_Turkish_Gra...
1032,1045,2020,15,3,Bahrain Grand Prix,2020-11-29,15:10:00,https://en.wikipedia.org/wiki/2020_Bahrain_Gra...
1033,1046,2020,16,3,Sakhir Grand Prix,2020-12-06,17:10:00,https://en.wikipedia.org/wiki/2020_Sakhir_Gran...


In [30]:
race_df.drop(columns=['round', 'circuitId', 'date', 'time', 'url'], inplace = True) 
race_df

,raceId,year,name
0,1,2009,Australian Grand Prix
1,2,2009,Malaysian Grand Prix
2,3,2009,Chinese Grand Prix
3,4,2009,Bahrain Grand Prix
4,5,2009,Spanish Grand Prix
...,...,...,...
1030,1043,2020,Emilia Romagna Grand Prix
1031,1044,2020,Turkish Grand Prix
1032,1045,2020,Bahrain Grand Prix
1033,1046,2020,Sakhir Grand Prix


In [31]:
df = df.merge(race_df, on = 'raceId', how = 'left')

In [32]:
df['Driver1'] = df['driverId']

In [33]:
df['Driver1_laptime'] = df['Fastest Lap Time']

In [34]:
df.rename(columns = {'Fastest Lap Time': 'Driver2_laptime'}, inplace= True)

In [35]:
df



,raceId,driverId,Rank,Driver2_laptime,Full Name,year,name,Driver1,Driver1_laptime
0,18,1,2,87.452,Lewis Hamilton,2008,Australian Grand Prix,1,87.452
1,18,2,3,87.739,Nick Heidfeld,2008,Australian Grand Prix,2,87.739
2,18,3,5,88.090,Nico Rosberg,2008,Australian Grand Prix,3,88.090
3,18,4,7,88.603,Fernando Alonso,2008,Australian Grand Prix,4,88.603
4,18,5,1,87.418,Heikki Kovalainen,2008,Australian Grand Prix,5,87.418
...,...,...,...,...,...,...,...,...,...
24855,1042,825,16,81.460,Kevin Magnussen,2020,Portuguese Grand Prix,825,81.460
24856,1042,154,17,81.664,Romain Grosjean,2020,Portuguese Grand Prix,154,81.664
24857,1042,849,19,81.859,Nicholas Latifi,2020,Portuguese Grand Prix,849,81.859
24858,1042,826,8,80.449,Daniil Kvyat,2020,Portuguese Grand Prix,826,80.449


In [36]:
rank_winners_2020 = [ 'Lewis Hamilton', 'Valtteri Bottas', 'Max Verstappen', 'Sergio Pérez', 'Charles Leclerc', 'Daniel Ricciardo', 
'Carlos Sainz', 'Lando Norris', 'Alexander Albon', 'Pierre Gasly', 'Lance Stroll', 'Esteban Ocon', 'Sebastian Vettel', 'Daniil Kvyat', 
'Nico Hülkenberg', 'Kimi Räikkönen', 'Antonio Giovinazzi', 'Romain Grosjean', 'Kevin Magnussen', 'Nicholas Latifi']
def name_check(stringy):
    return stringy in rank_winners_2020

In [37]:
new_df = df[df['Full Name'].map(name_check)]
new_df['Full Name']

0          Lewis Hamilton
7          Kimi Räikkönen
19       Sebastian Vettel
22         Kimi Räikkönen
26         Lewis Hamilton
               ...       
24855     Kevin Magnussen
24856     Romain Grosjean
24857     Nicholas Latifi
24858        Daniil Kvyat
24859        Lance Stroll
Name: Full Name, Length: 2536, dtype: object

In [38]:
new_df

,raceId,driverId,Rank,Driver2_laptime,Full Name,year,name,Driver1,Driver1_laptime
0,18,1,2,87.452,Lewis Hamilton,2008,Australian Grand Prix,1,87.452
7,18,8,4,87.903,Kimi Räikkönen,2008,Australian Grand Prix,8,87.903
19,18,20,\N,0.000,Sebastian Vettel,2008,Australian Grand Prix,20,0.000
22,19,8,2,95.405,Kimi Räikkönen,2008,Malaysian Grand Prix,8,95.405
26,19,1,3,95.462,Lewis Hamilton,2008,Malaysian Grand Prix,1,95.462
...,...,...,...,...,...,...,...,...,...
24855,1042,825,16,81.460,Kevin Magnussen,2020,Portuguese Grand Prix,825,81.460
24856,1042,154,17,81.664,Romain Grosjean,2020,Portuguese Grand Prix,154,81.664
24857,1042,849,19,81.859,Nicholas Latifi,2020,Portuguese Grand Prix,849,81.859
24858,1042,826,8,80.449,Daniil Kvyat,2020,Portuguese Grand Prix,826,80.449


In [39]:
new_df_gb = new_df.groupby(['Full Name', 'Driver1'])['Driver1_laptime', 'Driver2_laptime'].mean()
new_df_gb

<ipython-input-39-e0ea10064af1>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  new_df_gb = new_df.groupby(['Full Name', 'Driver1'])['Driver1_laptime', 'Driver2_laptime'].mean()


,,Driver1_laptime,Driver2_laptime
Full Name,Driver1,,
Alexander Albon,848,87.662303,87.662303
Antonio Giovinazzi,841,87.468343,87.468343
Carlos Sainz,832,84.740167,84.740167
Charles Leclerc,844,85.467019,85.467019
Daniel Ricciardo,817,90.965011,90.965011
Daniil Kvyat,826,89.946710,89.946710
Esteban Ocon,839,83.982758,83.982758
Kevin Magnussen,825,88.885435,88.885435
Kimi Räikkönen,8,73.335498,73.335498


In [40]:
new_df_gb = new_df_gb.sort_values(by= ['Driver1_laptime'], ascending= True)

In [41]:
new_df_gb = new_df_gb.round().astype(int)

In [42]:
new_df_gb.reset_index(inplace = True)
new_df_gb

,Full Name,Driver1,Driver1_laptime,Driver2_laptime
0,Kimi Räikkönen,8,73,73
1,Esteban Ocon,839,84,84
2,Pierre Gasly,842,85,85
3,Carlos Sainz,832,85,85
4,Nico Hülkenberg,807,85,85
5,Charles Leclerc,844,85,85
6,Max Verstappen,830,86,86
7,Lance Stroll,840,86,86
8,Romain Grosjean,154,86,86
9,Antonio Giovinazzi,841,87,87


In [43]:
new_df_gb.to_json('../data/processed/adj.json')

In [44]:
driver2_names = [ 'Nicholas Latifi', 'Lewis Hamilton', 'Valtteri Bottas', 'Max Verstappen', 'Sergio Pérez', 'Charles Leclerc', 'Daniel Ricciardo', 
'Carlos Sainz', 'Lando Norris', 'Alexander Albon', 'Pierre Gasly', 'Lance Stroll', 'Esteban Ocon', 'Sebastian Vettel', 'Daniil Kvyat', 
'Nico Hülkenberg', 'Kimi Räikkönen', 'Antonio Giovinazzi', 'Romain Grosjean', 'Kevin Magnussen']
driver1_names = [ 'Lewis Hamilton', 'Valtteri Bottas', 'Max Verstappen', 'Sergio Pérez', 'Charles Leclerc', 'Daniel Ricciardo', 
'Carlos Sainz', 'Lando Norris', 'Alexander Albon', 'Pierre Gasly', 'Lance Stroll', 'Esteban Ocon', 'Sebastian Vettel', 'Daniil Kvyat', 
'Nico Hülkenberg', 'Kimi Räikkönen', 'Antonio Giovinazzi', 'Romain Grosjean', 'Kevin Magnussen', 'Nicholas Latifi']
rank_dict = {'Driver1': driver1_names, 'Driver2': driver2_names }
rank_collection.update_one({}, {"$set": rank_dict}, upsert= True)

In [21]:
driver_df = pd.DataFrame(data = {'Driver1' : driver1_names})
driver_df

,Driver2
0,Lewis Hamilton
1,Valtteri Bottas
2,Max Verstappen
3,Sergio Pérez
4,Charles Leclerc
5,Daniel Ricciardo
6,Carlos Sainz
7,Lando Norris
8,Alexander Albon
9,Pierre Gasly


In [22]:
driver_df.to_json('../data/processed/adj.json')

In [ ]:
new_df_gb.drop(columns= 'Full Name', inplace = True)

In [ ]:
# driver_l = new_df_gb['Driver 1'].unique()
# driver_d = {'d1':0, 'd2':0}
# for d1 in driver_l:
#     result = 0
#     for d2 in driver_l:
#         temp = new_df_gb.loc[new_df_gb['Driver 1']== d1] & (new_df_gb.loc[new_df_gb['Driver1_laptime']== d1]) - new_df_gb.loc[new_df_gb['Driver 1']==d1] & (new_df_gb.loc[new_df_gbnew_df_gb['Driver2_laptim']== d2])
#         result = result + temp
#     driver_d[d1] = driver_d[d1] + result

In [ ]:
# race_l = new_df_gb['raceId'].unique()
# driver_l = new_df_gb['Driver 1'].unique()
# driver_d = {'d1':0, 'd2':0}

# for r in race_l:
#     for d1 in driver_l:
#         result = 0
#         for d2 in driver_l:
#             temp = df.loc[df['raceId']== r && df['Driver1_laptime']= d1] - df.loc[df['raceId']==r && df['Driver2_laptim']= d2]
#             result = result + temp
#         driver_d[d1] = driver_d[d1] + result
# data = pd.DataFrame(new_df_gb.to_numpy())
# data.astype(int)
data = pd.DataFrame(new_df_gb.to_numpy(), columns=['Driver 1', 'Driver1_laptime', 'Driver2_laptime'])
data.insert(0, 'Driver 2', range(0, 0 + len(data)), allow_duplicates=True)
data['Driver 1'] = data['Driver 2']
data

In [ ]:
d = [] 
d.append(data['Driver 2'].pop(data['Driver 2'].iloc[-1]))
for x in data['Driver 2']:
    d.append(x)
    

In [ ]:
dl = [] 
dl.append(data['Driver2_laptime'].iloc[-1])
for dx in data['Driver2_laptime']:
    dl.append(dx)
# .append(data['Driver2_laptime'])
dl.pop(-1)

In [ ]:
mdata = {'Driver 1': list(data['Driver 1']), 'Driver 2': d, 'Driver1_laptime': list(data['Driver1_laptime']), 'Driver2_laptime': dl}
gd = pd.DataFrame(mdata)
gd

In [ ]:
gd

In [ ]:
def init_linear_regressor_matrix(data, num_of_drivers, col_to_rank):
    
    """initialize linear system matrix for regression"""
    wins = np.zeros((data.shape[0], num_of_drivers))
    score_diff = np.zeros(data.shape[0])

    for index, row in data.iterrows():
        idx1 = row["Driver 1"]
        idx2 = row["Driver 2"]
        if row['Driver1_laptime'] - row['Driver2_laptime'] > 0:
            wins[(index)][(idx1)] = -1
            wins[(index)][(idx2)] = 1
            score_diff[(index)] = row['Driver1_laptime'] - row['Driver2_laptime']
        else:
            wins[(index)][(idx1)] = 1
        wins[(index)][(idx2)] = -1
        score_diff[(index)] = row['Driver2_laptime'] - row['Driver1_laptime']
    wins_df = pd.DataFrame(wins)
    wins_df[col_to_rank] = score_diff
    return wins_df 

def massey(data, num_of_drivers, col_to_rank='delta'):
    """Compute for each driver, adjacency matrix and aggregated scores, as input to the Massey Model"""

    wins_df = init_linear_regressor_matrix(data, num_of_drivers, col_to_rank)
    model = sm.OLS(
           wins_df[col_to_rank], wins_df.drop(columns=[col_to_rank])
       )
    results = model.fit(cov_type='HC1')
    rankings = pd.DataFrame(results.params)
    rankings['std'] = np.sqrt(np.diag(results.cov_params()))
    rankings['consistency'] = (norm.ppf(0.9)-norm.ppf(0.1))*rankings['std']
    rankings = (
         rankings
         .sort_values(by=0, ascending=False)
         .reset_index()
         .rename(columns={"index": "Driver", 0: "massey"})
     )
    rankings = rankings.sort_values(by=["massey"], ascending=False)
    rankings["massey_new"] = rankings["massey"].max() - rankings["massey"]
    return rankings[['Driver', 'massey_new']]

rankings = massey(gd, 10)
rankings
# print(rankings)

In [ ]:
# race_l = df['raceId'].unique()
# driver_l = df['Driver 1'].unique()
# driver_d = {'d1':0, 'd2':0}

# for r in race_l:
#     for d1 in driver_l:
#         result = 0
#         for d2 in driver_l:
#             temp = df.loc[df['raceId']== r && df['Driver 1']== d1] - df.loc[df['raceId']==r && df['Driver 2']== d2]
#             result = result + temp
#         driver_d[d1] = driver_d[d1] + result

In [ ]:
# df['Driver 2'] = df['Driver 2'].shift(-1)